In [1]:
import lammps
import random
from tqdm import trange
import pymc3 as pm
import arviz as az
import numpy as np
import pymatgen as pmg
import theano.tensor as tt
import glob

from poscar_to_lammps import poscar_to_lammps
from pymatgen.io.vasp import Poscar

import sys
from contextlib import contextmanager

In [2]:
@contextmanager
def custom_redirection(fileobj):
    old = sys.stdout
    sys.stdout = fileobj
    try:
        yield fileobj
    finally:
        sys.stdout = old

def structs_from_poscars(core_shell, charges):
    structures = []
    for i, structure in enumerate(glob.glob('poscars/POSCAR*')):
        file = 'lammps/coords{}.lmp'.format(i+1)
        with open(file, 'w') as struct:
            with custom_redirection(struct):
                poscar = Poscar.from_file(structure)
                poscar_to_lammps(poscar, core_shell, charges )
        structures.append(file)
    return structures

In [3]:
# 1. Define which elements are core-shell species
core_shell = { 'Li': False , 'Ni': False, 'O': True}

# 2. Set charges: dictionary values are either a single number for non-core-shell species, 
# or a sub-dictionary containing { 'core': core_charge, 'shell': shell_charge }
charges = {'Li': +1.0,
           'Ni': +3.0,
           'O': {'core':  +0.960,
                 'shell': -2.960}}
 # 3. Load a POSCAR or list of using pymatgen and appends to a list of structures
structures = structs_from_poscars(core_shell, charges)

elements = ['Li', 'Ni', 'Oc', 'O']
li, ni, oc, o = elements

pairs = [(li,o), (ni,o), (o,o)]

bpp_def = {'Li_O_A'   : 456.783,
           'Li_O_rho' : 0.212,
           'Li_O_C'   : 0.0,
           'Ni_O_A'   : 1540.922,
           'Ni_O_rho' : 0.323,
           'Ni_O_C'   : 0.425,
           'O_O_A'    : 22764.7000,
           'O_O_rho'  : 0.1490,
           'O_O_C'    : 20.4}

bpp = {'Li_O_A'   : 456.783,
       'Li_O_rho' : 0.212,
       'Li_O_C'   : 0.0,
       'Ni_O_A'   : 1540.922,
       'Ni_O_rho' : 0.323,
       'Ni_O_C'   : 0.425,
       'O_O_A'    : 22764.7000,
       'O_O_rho'  : 0.1490,
       'O_O_C'    : 20.4}


In [4]:
def init(structure):
    lmp = lammps.Lammps(units='metal', style = 'full', args=['-log', 'none', '-screen', 'none'])
    lmp.command('fix csinfo all property/atom i_CSID')
    lmp.command('read_data {} fix csinfo NULL CS-Info'.format(structure))

    lmp.command('group cores type 3')
    lmp.command('group shells type 4')

    lmp.command('pair_style buck/coul/long/cs 12.0') #ADD THE CS IF USING CORESHELLS NOT IF RIGID ION
    lmp.command('pair_coeff * * 0 1 0')
    
    lmp.command('bond_style harmonic')
    lmp.command('bond_coeff 1 65.0 0.0')
    
    lmp.command('kspace_style pppm 1e-6')

        
    #setup for minimization
    lmp.command('min_style cg')

    return lmp

In [5]:
def update_potentials(**kwargs):
    bpp.update(kwargs)
    
def set_potentials(instance):
    for pair in pairs:
        instance.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]),
                                                            dex(pair[1]),
                                                            bpp['{}_{}_A'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_C'.format(pair[0], pair[1])]))

def dex(elin): return elements.index(elin) + 1

In [6]:
def simfunc(**kwargs):
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        out = np.zeros([instances[0].system.total,3, len(instances)])
        
        for instance in instances:
            set_potentials(instance)
            instance.command('fix 1 cores setforce 0.0 0.0 0.0')
            instance.command('minimize 1e-5 1e-5 5000 10000')
            instance.command('unfix 1')
            instance.run(0)            
            out[:,:,instances.index(instance)] = instance.system.forces
            
    else: out = np.ones([instances[0].system.total,3, len(instances)])*999999999 # ThisAlgorithmBecomingSkynetCost
        
    return out

In [7]:
instances = [init(structure) for structure in structures]

In [ ]:
expected = np.zeros([instances[0].system.total,3, len(instances)])

In [ ]:
with pm.Model() as model:
#  pm.TruncatedNormal -- truncated so never tries a negative    pm.Uniform
    Li_O_A      = pm.Normal("Li_O_A",    mu = bpp_def['Li_O_A'],    sd =  100)
    Li_O_rho    = pm.Normal("Li_O_rho",  mu = bpp_def['Li_O_rho'],  sd =  0.05)
#     Li_O_C      = pm.Normal("Li_O_C",    mu = bpp_def['Li_O_C'],    sd =  0.01)
    Ni_O_A      = pm.Normal("Ni_O_A",    mu = bpp_def['Ni_O_A'],    sd =  100)
    Ni_O_rho    = pm.Normal("Ni_O_rho",  mu = bpp_def['Ni_O_rho'],  sd =  0.05)
    Ni_O_C      = pm.Normal("Ni_O_C",    mu = bpp_def['Ni_O_C'],    sd =  0.05)
    O_O_A       = pm.Normal("O_O_A",     mu = bpp_def['O_O_A'],     sd =  100)
    O_O_rho     = pm.Normal("O_O_rho",   mu = bpp_def['O_O_rho'],   sd =  0.05)
    O_O_C       = pm.Normal("O_O_C",     mu = bpp_def['O_O_C'],     sd =  5)
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=0.1), draws=1000)


In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(trace)

In [ ]:
az.plot_posterior(trace, round_to = 3, point_estimate = 'mode')

In [ ]:
structures